---
title: 'News API'
---

In [1]:
import requests
import json
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

baseURL = "https://newsapi.org/v2/everything?"
total_requests=2
verbose=True

API_KEY='2ce6e84cac934909813dcebe5bcb2eca'
TOPIC=['fast fashion']

topic_dict = {}

for topic in TOPIC:
    URLpost = {'apiKey': '2ce6e84cac934909813dcebe5bcb2eca',
        'q': topic,
        'sortBy': 'relevancy',
        'totalRequests': 1}
    print(baseURL)
    response = requests.get(baseURL, URLpost) 
    response = response.json()
    print(json.dumps(response, indent=2))
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y-%m-%d-H%H-M%M-S%S")

    def string_cleaner(topic):
        try: 
            out=re.sub(r"""
                    [,.;@#?!&$-]+  
                    \ *           
                    """,
                    " ",          
                    topic, flags=re.VERBOSE)
            out = re.sub('[’.]+', '', topic)
            out = re.sub(r'\s+', ' ', out)
            out=out.lower()
        except:
            print("ERROR")
            out=''
        return out
    
    with open(timestamp+'-newapi-raw-data'+topic+'.json', 'w') as outfile:
        json.dump(response, outfile, indent=4)

    articles = response["articles"]
    text_data = [{"title": article["title"], "description": article["description"]} for article in articles]
    topic_dict[topic] = text_data

    article_list=response['articles']   #list of dictionaries for each article
    article_keys=article_list[0].keys()
    print("AVAILABLE KEYS:")
    print(article_keys)
    index=0
    cleaned_data=[];  
    for article in article_list:
        tmp=[]
        if(verbose):
            print("#------------------------------------------")
            print("#",index)
            print("#------------------------------------------")

        for key in article_keys:
            if(verbose):
                print("----------------")
                print(key)
                print(article[key])
                print("----------------")

            if(key=='source'):
                src=string_cleaner(article[key]['name'])
                tmp.append(src) 

            if(key=='author'):
                author=string_cleaner(article[key])
                if(src in author): 
                    print(" AUTHOR ERROR:",author);author='NA'
                tmp.append(author)

            if(key=='title'):
                tmp.append(string_cleaner(article[key]))

            if(key=='publishedAt'):
                ref = re.compile('.*-.*-.*T.*:.*:.*Z')
                date=article[key]
                if(not ref.match(date)):
                    print(" DATE ERROR:",date); date="NA"
                tmp.append(date)

        cleaned_data.append(tmp)
        index+=1
    
    df = pd.DataFrame(cleaned_data)
    print(df)
    df.to_csv('cleaned'+topic+'.csv', index=False)

https://newsapi.org/v2/everything?
{
  "status": "ok",
  "totalResults": 1753,
  "articles": [
    {
      "source": {
        "id": "time",
        "name": "Time"
      },
      "author": "C\u00e9line Semaan",
      "title": "What Needs to Happen to Tackle Fashion\u2019s Climate Impact",
      "description": "Is the fashion industry anxious enough enough to combat climate change, asks C\u00e9line Semaan.",
      "url": "https://time.com/6318970/fashion-climate-impact-anxiety-essay/",
      "urlToImage": "https://api.time.com/wp-content/uploads/2023/09/chile-clothing-dump.jpg?quality=85",
      "publishedAt": "2023-09-30T11:00:00Z",
      "content": "On Sept. 17, on the heels of New York Fashion Week, Climate Week saw more than 70,000 people marched in the streetsof Manhattan demanding the end of fossil fuel industries and climate justice at scal\u2026 [+7996 chars]"
    },
    {
      "source": {
        "id": null,
        "name": "Fashionista"
      },
      "author": "Shelby Ying H